# Suraj Pawar 
## A20163517


# Project-2: Locality Sensitive Hashing

In [ ]:
import random
import os
import numpy as np
import collections

from random import randrange
from shutil import copyfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Execute the follwing two cells to generate your data sample

In [ ]:
MY_ID = '517'
# MY_ID should be string
# Example MY_ID = '819'

task_dict = {0:'taska',1:'taskb',2:'taskc',3:'taskd',4:'taske'}

n_1 = 0
n_2 = 0
n_3 = 0

try:
    n_1 = int(MY_ID[-1]) % 5
    n_2 = int(MY_ID[-2]) % 5
    n_3 = int(MY_ID[-3]) % 5
    
    while n_1 == n_2 or n_1 == n_3:
        n_1 = (n_1 + 1) % 5
    
    while n_1 == n_2 or n_2 == n_3:
        n_2 = (n_2 + 1) % 5

except Exception as e:
    print('Please enter a valid ID...')

id_dict = {0:n_1,1:n_2,2:n_3}



In [ ]:
data_path = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-2/corpus-20090418' 
# Edit this path if the data directory is not in the current directory

try:
    os.makedirs('Data_Sample')
    os.makedirs('Original_Sample')
except Exception as e:
    pass

for i in range(3):
    task = task_dict[id_dict[i]]
    
    for file in os.listdir(data_path):
        if task in file:
            if 'orig' in file:
                copyfile(data_path + '/' + file, 'Original_Sample/' + file)
            else:
                copyfile(data_path + '/' + file, 'Data_Sample/' + file)

Your dataset for this project will be in <b>Data_Sample</b>


Your query choices will be in <b>Original_Sample</b> directory


You have to use any one original Wikipedia article from <b>Original_Sample</b> for <b>Fact Check</b> steps

### STEP - 1: Shingling (20 points)

In [ ]:
# Type your code here... 
# Create necessary number of cells below this cell
k1 = 3
k2 = 4
k3 = 5

# NOTE: No complex text processing is required
# convert just upper case characters to lower case


In [ ]:
dir_path = 'Original_Sample'

doc_lines = []
counter = 0
for filename in os.listdir(dir_path):
  counter = counter + 1
  file = open(dir_path + '/' + filename, 'r',encoding='utf-8',errors='ignore')
  lines = file.read()
  doc_lines.append(lines)

dir_path = 'Data_Sample'
for filename in os.listdir(dir_path):
  counter = counter + 1
  file = open(dir_path + '/' + filename, 'r',encoding='utf-8',errors='ignore')
  lines = file.read()
  doc_lines.append(lines)

n_docs = len(doc_lines)


In [ ]:
shingles_dict = {}
for k in [k1,k2,k3]:
  shingles = []
  for doc_line in doc_lines:
    l = doc_line.lower().strip().split()
    for i in range(len(l)-k+1):
      k_sh = tuple(l[i:i+k])
      shingles.append(k_sh)

  shingles_set = set(shingles)
  shingles_set_list = list(shingles_set)
  shingles_set_list = [' '.join(item) for item in shingles_set_list]
  shingles_dict[k] = shingles_set_list

  print(f'k = {k} : {len(shingles_dict[k])}')



k = 3 : 8086
k = 4 : 8848
k = 5 : 9220


Report results (number of unique k-shingles) for k={3,4,5} below:
1. k=3: 8977
2. k=4: 9793
3. k=5: 10169

### STEP - 2: Min-Hashing (40 points)

In [ ]:
# ***************************************************NEW***********************************************************
# Generate Hash functions - 
    # We use (ax + b) mod N formula to permute shingle index
    # where a,b are random numbers, N total index size, and x is the index
# We need to do L permutations - In other words we need to have L permutations (lists) of new indexes
# Following function takes total index size N and L as arguments
    # And returns L new lists of size N
    
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions
        
# test
# hash_functions = get_hash_functions(N,50)

In [ ]:
n_docs_data = n_docs - 3

n = 0
file_name_dict = {}

dir_path = 'Data_Sample'
file_shingles_dict = {}

for file in os.listdir(dir_path):
  file_name_dict[n] = file
  file = open(dir_path + '/' + file, 'r',encoding='utf-8',errors='ignore')
  
  doc_shingles = []

  data = file.read()
  l = data.lower().strip().split()
  
  for i in range(len(l)-k+1):
    k_sh = tuple(l[i:i+k])
    doc_shingles.append(k_sh)
  doc_shingles = [' '.join(item) for item in doc_shingles]
  file_shingles_dict[n] = doc_shingles
  n = n + 1

In [ ]:
# Type your code here to generate all L hash functions
# Generate hash functions only for shingle index created for k=5
k = k3
shingles_k5 = shingles_dict[5]
N = len(shingles_k5)

In [ ]:
for L in [50,100,200,500,1000]:
#for L in [50]:
  hash_functions = get_hash_functions(N,L)
  M = np.full((L,n_docs_data), np.inf)

  for key,doc_shingles in file_shingles_dict.items():
    for j in range(N):
      if shingles_k5[j] in doc_shingles:
        for i in range(L):
          if hash_functions[i][j] < M[i,key]:
            M[i,key] = hash_functions[i][j]

  print(M)
  
  # print(f'The shape of the signature matrix is [{M.shape[0]}, {M.shape[1]}]')

  # Type your code here to do the fact check 
  #      with any one query document in the 'Original_Sample' directory

  original = 'Original_Sample/orig_taska.txt'

  # STEP-1: Generate 5-shingles 
      # (if any shingles are no t present in your shingle index, simply ignore them)

  S = np.zeros(N)

  file = open(original, 'r',encoding='utf-8',errors='ignore')
  data = file.read()

  doc_shingles = []
  l = data.lower().strip().split()
  for i in range(len(l)-k+1):
    k_sh = tuple(l[i:i+k])
    doc_shingles.append(k_sh)

  doc_shingles = [' '.join(item) for item in doc_shingles]
      
  # STEP-2: Generate signature vector from L hash functions
  Mo = np.full((L,1), np.inf) 
  for j in range(N):
    if shingles_k5[j] in doc_shingles:
      for i in range(L):
        if hash_functions[i][j] < Mo[i,0]:
          Mo[i,0] = hash_functions[i][j]  


  # STEP-3: Calculate Jaccard similarity of signature vector of orginal doc.
      # and all other documents 
  t = 0.2
  similar_docs = {}
  for i in range(n_docs_data):
    same = M[:,i] == Mo[:,0]
    count = sum(same)/L
    if count >= t:
      similar_docs[file_name_dict[i]] = count*100
      # print(i, ' ', file_name_dict[i], ' ', count*100)
  
  similar_docs_sorted = {k: v for k, v in sorted(similar_docs.items(), key=lambda item: item[1])}

  print ("{:<5} {:<25} {:<10} ".format('L', 'Name of the document', 'Jaccard similarity'))
  for key,value in similar_docs_sorted.items():
    print ("{:<5} {:<25} {:<10} ".format(L, key, round(value,2)))
    # print(L, ' ', key, ' ', round(value,2))
  
  print('---------------------------------------------------')


L     Name of the document      Jaccard similarity 
50    g0pD_taska.txt            20.0       
50    g2pE_taska.txt            26.0       
50    g2pC_taska.txt            32.0       
50    g3pC_taska.txt            42.0       
50    g4pC_taska.txt            84.0       
50    g0pE_taska.txt            88.0       
---------------------------------------------------
L     Name of the document      Jaccard similarity 
100   g0pD_taska.txt            20.0       
100   g2pC_taska.txt            26.0       
100   g3pC_taska.txt            30.0       
100   g4pC_taska.txt            85.0       
100   g0pE_taska.txt            89.0       
---------------------------------------------------
L     Name of the document      Jaccard similarity 
200   g2pC_taska.txt            26.5       
200   g0pD_taska.txt            28.5       
200   g3pC_taska.txt            38.5       
200   g4pC_taska.txt            87.0       
200   g0pE_taska.txt            92.0       
------------------------------------

***************************************************NEW***********************************************************
For each L = {50,100,200,500,1000}, report all documents (file_names) below that have Jaccard similarity > t=0.2
Sort the documents in decreasing order of the Jaccard similarity

Below are the results I got for one of the run. Results might be little different based on the random number used in hash functions. 

---------------------------------------------------

L     Name of the document      Jaccard similarity 

50    g3pC_taska.txt            22.0       
50    g0pD_taska.txt            26.0       
50    g2pC_taska.txt            30.0       
50    g4pC_taska.txt            88.0       
50    g0pE_taska.txt            92.0      

---------------------------------------------------
L     Name of the document      Jaccard similarity 

100   g0pD_taska.txt            28.0       
100   g3pC_taska.txt            33.0       
100   g2pC_taska.txt            35.0       
100   g4pC_taska.txt            90.0       
100   g0pE_taska.txt            92.0    

---------------------------------------------------
L     Name of the document      Jaccard similarity 

200   g3pC_taska.txt            28.5       
200   g0pD_taska.txt            29.0       
200   g2pC_taska.txt            30.5       
200   g4pC_taska.txt            85.0       
200   g0pE_taska.txt            89.0     

---------------------------------------------------
L     Name of the document      Jaccard similarity 

500   g0pD_taska.txt            24.4       
500   g3pC_taska.txt            32.0       
500   g2pC_taska.txt            37.6       
500   g4pC_taska.txt            84.4       
500   g0pE_taska.txt            88.6   

---------------------------------------------------
L     Name of the document      Jaccard similarity 

1000  g0pD_taska.txt            24.2       
1000  g3pC_taska.txt            32.7       
1000  g2pC_taska.txt            35.2       
1000  g4pC_taska.txt            83.9       
1000  g0pE_taska.txt            89.3       



### STEP - 3: LSH (30 points)

In [ ]:
# Type your code here to hash signature matrix into B buckets
# Use the technique to split the signature matrix into b bands of r rows
# Convert only the signature matrix generated with L=1000

b = 50
r = 20
B = 199
t = 0.2

len_sim_docs = len(similar_docs_sorted)

ar = random.sample(range(100), r)
# print(ar)

ar = np.array(ar)

In [ ]:
# Type your code here to do generate candidate documents
# Follow all steps from STEP - 2 fact check (except the Jaccard similarity part)

# STEP - 1: Split your original document signature vector into b bands of r rows

bucket = collections.defaultdict(set)

Mb = np.reshape(M,[b,r,n_docs_data]) 

# STEP - 2: Hash using the same hash functions created for 
    # signature matrix hashing (in the previous cell)

for i in range(b):
  dict_band = collections.defaultdict(list)
  
  for n in range(n_docs_data):
    x = Mb[i,:,n]
    h = (np.sum(ar*x)) % B
    dict_band[h].append(file_name_dict[n])
  
  for j in dict_band:
    if len(dict_band[j]) >= 2:
      for item in dict_band[j]:
        bucket[j].add(item)


In [ ]:
# Type your code here to do the fact check
# Calculate Jaccard similarity of the oiginal document with only 
    # candidate documents

original = 'Original_Sample/orig_taska.txt'

# STEP-1: Generate 5-shingles 
    # (if any shingles are no t present in your shingle index, simply ignore them)

file = open(original, 'r',encoding='utf-8',errors='ignore')

data = file.read()

doc_shingles = []
l = data.lower().strip().split()
for i in range(len(l)-k+1):
  k_sh = tuple(l[i:i+k])
  doc_shingles.append(k_sh)

doc_shingles = [' '.join(item) for item in doc_shingles]
    
# STEP-2: Generate signature vector from L hash functions
Mo = np.full((L,1), np.inf) 

for j in range(N):
  if shingles_set_list[j] in doc_shingles:
    for i in range(L):
      if hash_functions[i][j] < Mo[i,0]:
        Mo[i,0] = hash_functions[i][j] 

In [ ]:
bucket_o = collections.defaultdict(set)

Mb_o = np.reshape(Mo,[b,r,1]) 

# STEP - 2: Hash using the same hash functions created for 
    # signature matrix hashing (in the previous cell)

for i in range(b):
  dict_band = collections.defaultdict(list)
  
  x = Mb_o[i,:,0]
  h = (np.sum(ar*x)) % B
  bucket_o[h].add(i)

dict_check = collections.defaultdict(set)

for i in bucket_o:
  for item in bucket[i]:
    dict_check[0].add(item)

lsh_list = []
file_name_dict_swap = {value:key for key, value in file_name_dict.items()}

for item in dict_check[0]:
  lsh_list.append(file_name_dict_swap[item])

print(f'Number of candidate documents is {len(lsh_list)}')

Number of candidate documents is 55


In [ ]:
# STEP-3: Calculate Jaccard similarity of signature vector of orginal doc.
    # and all other documents 
M_lsh = M[:,lsh_list]
similar_docs_lsh = {}

for i in range(M_lsh.shape[1]):
  same = M_lsh[:,i] == Mo[:,0]
  count = sum(same)/L
  if count >= t:
    similar_docs_lsh[file_name_dict[lsh_list[i]]] = count*100
    # print(i, ' ', lsh_list[i], ' ', file_name_dict[lsh_list[i]], ' ', count*100)

similar_docs_lsh_sorted = {k: v for k, v in sorted(similar_docs_lsh.items(), key=lambda item: item[1])}

print ("{:<10} {:<25} {:<10} ".format('L', 'Name of the document', 'Jaccard similarity'))
for key,value in similar_docs_lsh_sorted.items():
  print ("{:<10} {:<25} {:<10} ".format(L, key, round(value,2)))



L          Name of the document      Jaccard similarity 
1000       g0pD_taska.txt            21.0       
1000       g3pC_taska.txt            33.7       
1000       g2pC_taska.txt            34.1       
1000       g4pC_taska.txt            84.2       
1000       g0pE_taska.txt            87.8       


Report all documents (file_names) below that have Jaccard similarity > t=0.85
Sort the documents in decreasing order of the Jaccard similarity

L          Name of the document      Jaccard similarity 

1000       g0pD_taska.txt            24.2       
1000       g3pC_taska.txt            32.7       
1000       g2pC_taska.txt            35.2       
1000       g4pC_taska.txt            83.9       
1000       g0pE_taska.txt            89.3 



Report the list of false positives and false negatives below



In [ ]:
false_positives = len(lsh_list) - len(similar_docs_lsh_sorted)
false_negatives = len_sim_docs - len(similar_docs_lsh_sorted)

print(f'Number of false positives = {false_positives}')
print(f'Number of false negatives = {false_negatives}')

fp_list = []
fn_list = []

similar_docs_list = [key for key,value in similar_docs_sorted.items()]
similar_docs_lsh_list = [key for key,value in similar_docs_lsh_sorted.items()]

for key in dict_check[0]:
  if key not in similar_docs_list:
    fp_list.append(key)

for key in similar_docs_list:
  if key not in dict_check[0]:
    fn_list.append(key)

Number of false positives = 50
Number of false negatives = 0


In [ ]:
print('The list of false positives is as below : ')
for i in fp_list:
  print(i)

The list of false positives is as below : 
g3pB_taskb.txt
g3pA_taskc.txt
g0pE_taskc.txt
g4pD_taskb.txt
g2pE_taskb.txt
g2pE_taska.txt
g1pB_taskb.txt
g0pC_taskb.txt
g0pE_taskb.txt
g4pC_taskc.txt
g0pA_taskb.txt
g4pB_taska.txt
g4pD_taskc.txt
g3pB_taskc.txt
g3pC_taskb.txt
g4pD_taska.txt
g1pB_taskc.txt
g2pE_taskc.txt
g3pC_taskc.txt
g4pE_taskb.txt
g1pD_taska.txt
g2pB_taska.txt
g0pD_taskb.txt
g0pC_taskc.txt
g4pE_taskc.txt
g2pB_taskb.txt
g2pC_taskc.txt
g1pD_taskb.txt
g1pA_taskb.txt
g0pD_taskc.txt
g0pC_taska.txt
g1pB_taska.txt
g0pB_taskb.txt
g2pC_taskb.txt
g2pB_taskc.txt
g1pA_taskc.txt
g4pC_taskb.txt
g0pB_taskc.txt
g4pB_taskb.txt
g4pE_taska.txt
g0pA_taska.txt
g0pB_taska.txt
g2pA_taska.txt
g2pA_taskb.txt
g4pB_taskc.txt
g0pA_taskc.txt
g2pA_taskc.txt
g3pB_taska.txt
g1pA_taska.txt
g1pD_taskc.txt


In [ ]:
print('The list of false negatives is as below : ')
for i in fn_list:
  print(i)

The list of false negatives is as below : 


### BONUS : 10 points

In [ ]:
# Experiment with different values of b,r,B, and t
    # to reduce the number of false positives and false negatives
    # Report all results in a table in a separate word document